In [1]:
import pandas as pd
import yfinance as yf

# Børsnoterede virksomheder

In [2]:
df_virk = pd.read_excel("børsnoterede_virk.xlsx")

In [3]:
df_virk.dropna(inplace=True)

In [4]:
df_virk.head()

,Navn,Symbol,ISIN,ICB Code
1,Ørsted,ORSTED,DK0060094928,6510.0
2,Össur,OSSR,IS0000000040,2010.0
3,Zealand Pharma,ZEAL,DK0060257814,2010.0
4,Victoria Properties,VIPRO,DK0015216675,3510.0
5,Vestjysk Bank,VJBA,DK0010304500,3010.0


In [5]:
# omdan symbol til international ticker
tickers = []
for i in df_virk.Symbol:
    ticker = i+".CO"
    tickers.append(ticker)

In [6]:
# hent aktie data
data = yf.download(tickers=tickers,\
                  period ="max",\
                  interval = "1d",\
                  group_by = 'ticker',\
                  auto_adjust = True
                  )

[*********************100%***********************]  131 of 131 completed

1 Failed download:
- BOOZT-DKK.CO: 1d data not available for startTime=-2208988800 and endTime=1616331911. Only 100 years worth of day granularity data are allowed to be fetched per request.


In [7]:
# strukturer data
price_df = pd.DataFrame()
for i in tickers:
    price_df = pd.concat([price_df,data[str(i)].Close],axis=1)

price_df.columns = tickers

# long-format dataframe

In [8]:
list_of_dfs = []
for i in price_df:
    df = pd.DataFrame()
    df["dato"] = price_df[str(i)].index
    df["ticker"] = i

    # gendan til dansk ticker
    symbol = []
    for tick in df["ticker"]:
        symbol.append(tick.rstrip(".CO"))
    df["symbol"] = symbol
    
    # merge data for at få firma navn
    df = df.merge(df_virk,left_on="symbol",right_on="Symbol").drop(["Symbol","ISIN","ICB Code"],axis=1).rename(columns={"Navn":"firma"})
    
    # tilføj pris-data
    df["adj_close"]=price_df[str(i)].tolist()
    
    list_of_dfs.append(df)

In [9]:
df = pd.concat(list_of_dfs)

In [10]:
df_virk[65:].Navn.tolist()

['Københavns Lufthavne',
 'Kreditbanken',
 'Jyske Bank',
 'Jutlander Bank',
 'Jeudan',
 'ISS',
 'InterMail',
 'Hvidbjerg Bank',
 'HusCompagniet',
 'Harboes Bryggeri B',
 'H+H International',
 'Gyldendal B',
 'Gyldendal A',
 'Grønlandsbanken',
 'GreenMobility',
 'GN Store Nord',
 'Glunz & Jensen Holding',
 'German High Street Properties',
 'Genmab',
 'Gabriel Holding',
 'G4S plc',
 'Fynske Bank',
 'Flügger group B',
 'FLSmidth & Co.',
 'FirstFarms',
 'Fast Ejendom Danmark',
 'EAC Invest',
 'DSV Panalpina',
 'Drilling Company of 1972',
 'Djurslands Bank',
 'DFDS',
 'Demant',
 'Dantax',
 'Danske Bank',
 'Danske Andelskassers Bank',
 'D/S Norden',
 'Copenhagen Capital Stam',
 'Copenhagen Capital Præf',
 'Columbus',
 'Coloplast B',
 'Chr. Hansen Holding',
 'ChemoMetec',
 'Cemat',
 'CBRAIN',
 'Carlsberg B',
 'Carlsberg A',
 'Brøndby IF',
 'Brdr.Hartmann',
 'Brdr. A & O Johansen præf.',
 'Brd. Klee B',
 'Boozt',
 'Boliga Gruppen',
 'Blue Vision A',
 'BioPorto',
 'Bavarian Nordic',
 'BankNordi

# Rens virksomheder

In [11]:
df_komp = pd.read_csv("UDFYLD_firm.csv",sep="\t")

/Users/smeden/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [18]:
df_komp[df_komp.firma.str.contains('Brøndby IF' , case=False, regex=False) == True]

,Unnamed: 0,ID,firma,match_cvr,prob_cvr,match_cvr_virknavn,match_cvr_branche,match_cvr_ansatte_str,frivillig_job
1555823,1555823,k20140212h4435212359,Brøndby IF,65113015,0.577148,BRØNDBY KOMMUNE,841100.0,ANTAL_1000_999999,alle andre
1874361,1874361,k2013090151741341156,Brøndby If,65113015,0.747559,BRØNDBY KOMMUNE,841100.0,ANTAL_1000_999999,frivilligjob
2219957,2219957,k20110729h3212072733,Brøndby IF,65113015,0.573242,BRØNDBY KOMMUNE,841100.0,ANTAL_1000_999999,alle andre
2319256,2319256,k2010022634184841863,Brøndby IF Arrangementer,65113015,0.737305,BRØNDBY KOMMUNE,841100.0,ANTAL_1000_999999,alle andre
2362184,2362184,k20100601h2703741955,Brøndby IF,65113015,0.556641,BRØNDBY KOMMUNE,841100.0,ANTAL_1000_999999,alle andre
2449936,2449936,k20101215h2918082407,Brøndby IF,65113015,0.577148,BRØNDBY KOMMUNE,841100.0,ANTAL_1000_999999,alle andre
2479495,2479495,k20090213h2320323324,Brøndby IF,21479578,0.426514,alles Lægehus A/S,862100.0,ANTAL_200_499,alle andre
2602617,2602617,k20091111h2520543043,Brøndby IF,65113015,0.743164,BRØNDBY KOMMUNE,841100.0,ANTAL_1000_999999,alle andre
2645261,2645261,k20080122h1747500063,Brøndby IF,65113015,0.781738,BRØNDBY KOMMUNE,841100.0,ANTAL_1000_999999,alle andre
2836218,2836218,k20080805h2082251602,Brøndby IF,65113015,0.573242,BRØNDBY KOMMUNE,841100.0,ANTAL_1000_999999,alle andre


In [12]:
df_komp[df_komp.firma.str.contains('aab a/S' , case=False, regex=False) == True].drop_duplicates('ID').firma.unique().tolist()

['AaB A/S', 'Daab A/S', 'Daab a/s']

In [26]:
Københavns_Lufthavne_list = ['Københavns Lufthavne A/S'
                             ,'Københavns Lufthavn'
                             ,'Københavns Lufthavne (CPH)'
                             ,"Københavns lufthavne"
                             ,'Copenhagen Airport Hotels A/S'
                             ,'COPENHAGEN AIRPORT HOTEL ApS'
                             ,"COPENHAGEN AIRPORTS INTERNATIONAL A/S"
                             ,"Københavns Lufthavne A/S, Hrm"]

Kreditbanken_list = ['Kreditbanken A/S'
               ,'Kreditbanken A/S Sønderborg'
               ,'Kreditbanken A/S Tønder'
               ,'Kreditbanken'
               ,'Kreditbanken i Sønderborg'
               ,'KREDITBANKEN A/S']


Jyske_Bank_list = ['Jyske Bank'
                  ,'Jyske Finans'
                  ,'JYSKE FINANS A/S'
                  ,'Jyske Invest']

Jutlander_Bank_list = ['Jutlander Bank A/S'
                      ,'Jutlander Bank'
                      ,'Jutlander Bank AS'
                      ,'JUTLANDER BANK A/S'
                      ,' Jutlander Bank A/S']

Jeudan_list = ['Jeudan A/S'
              ,'Jeudan'
              ,'JEUDAN A/S'
              ,'Jeudan Servicepartner A/S'
              ,'JEUDAN SERVICEPARTNER A/S'
              ,'Jeudan Servicepartner']

ISS_list = ['ISS A/S'
           ,'ISS World Services A/S'
           ,'ISS World'
           ,'ISS Facility Services A/S'
           ,'ISS Facility Services AS'
           ,'Personalehuset - Part of ISS Facility Services A/S'
           ,'HR afd. i ISS Facility Services A/S'
           ,'Medarbejderfond for ansatte i ISS Facility Services A/S'
           ,'ISS FACILITY SERVICES A/S'
           ,'ISS Facility Services A/S Fredericia Sygehus'
           ,'ISS Facility Services'
           ,'ISS Facility Service'
           ,'ISS Facility Service A/S Rexam Galss Holmegaard A/S'
           ,'Iss Facility Services A/S'
          ,'ISS Facility Services A/S Rødby Færge'
         ,'ISS Facility Services A/S CSC Scandihealth A/S'
         ,'ISS Facility Services A/S Rexam Glass Holmegaard A/S'
         ,'ISS Facility Services A/S Salg'
         ,'ISS Facility Services A/S - El-Partners'
         ,'ISS Facility Services A/S Kolding Sygehus'
         ,'ISS Facility Services A/S Ringsted Sygehus'
         ,'Hørsholm Hospital, ISS Facility Services'
        ,'Hørsholm Hospital, ISS Facility Services'
         ,'ISS Facility Services A/S Office Support'
         ,'ISS Facillity Services A/S'
         ,'ISS Facility Service A/S'
         ,'ISS Facility Services A/S Sønderborg Sygehus'
         ,'ISS Facility Services A/S Aabenraa Sygehus'
         ,'ISS Facility Services A/S Bilka'
         ,'ISS Facility Services A/S Haderslev sygehus'
         ,'Iss Facility Services'
           ,'ISS Services A/S']

InterMail_list = ['Intermail Danmark A/S'
                 ,'InterMail'
                 ,'InterMail A/S'
                 ,'InterMail Solutions A/S'
                 ,'Intermail Solutions']

Hvidbjerg_Bank_list = ['Hvidbjerg Bank'
                      ,'Hvidbjerg Bank Aktieselskab'
                      ,'Hvidbjerg Bank Holstebro'
                      ,'HVIDBJERG BANK'
                      ,'HVIDBJERG BANK A/S'
                      ,'Hvidbjerg bank']

HusCompagniet_list = ['HusCompagniet A/S'
                    ,'Huscompagniet Midt- og Nordjylland A/S'
                     ,'HusCompagniet'
                     ,'HusCompagniet Sjælland A/S'
                     ,'HusCompagniet Midt- og Nordjylland A/S'
                     ,'HusCompagniet Fyn A/S'
                     ,'Huscompagniet Sjælland A/S'
                     ,'Huscompagniet A/S'
                     ,'HusCompagniet as'
                     ,'HusCompagniet Midt- og Nordjylland'
                     ,'HusCompagniet Sønderjylland as']

Harboes_Bryggeri_list = ['Harboes Bryggeri A/S'
                          ,'HARBOES BRYGGERI A/S']

H_H_International_list = ['H+H International A/S'
                         ,'H+H Danmark A/S'
                         ,'H+H EJENDOMME A/S']

Gyldendal_list = ['Gyldendals Bogklubber'
                 ,'Gyldendal A/S'
                 ,'Gyldendal'
                 ,'GYLDENDAL'
                 ,'GYLDENDAL A/S'
                 ,'Forlaget Gyldendal A/S'
                 , 'Gyldendal Uddannelse'
                 , 'Gyldendal Business'
                 ,'Gyldendal Akademisk A/S'
                 ,'Gyldendal Akademisk'
                 ,'Gyldendal Digital']

Grønlandsbanken_list = ['Grønlandsbanken A/S'
                 ,'Grønlandsbanken Qaqortoq afdeling'
                 ,'GrønlandsBANKEN'
                 ,'Grønlandsbanken'
                 ,'GrønlandsBANKEN A/S']


GreenMobility_list = ['GreenMobility'
                      ,'GreenMobility A/S'
                     ,'Green Mobility A/S']

GN_Store_Nord_list = ['GN Store Nord',
                     'GN Store Nord A/S',
                     'GN STORE NORD A/S',
                     'GN Store Nord as',
                     'Gn Store Nord A/S'
                     'GN AUDIO A/S', 'GN Audio A/S', 'GN Audio'
                     'GN FalCom'
                     'GN Hearing A/S', 'GN HEARING A/S', 'GN Hearing DK A/S', 'GN Hearing',
                     'Dansk HøreCenter',
                     'Dansk Hørecenter',
                     'DANSK HØRETEKNIK A/S']

Glunz_and_Jensen_Holding_list = ['Glunz & Jensen A/S',
 'Glunz & Jensen Microflex A/S',
 'Glunz & Jensen',
 'GLUNZ & JENSEN A/S',
 'GLUNZ & JENSEN MICROFLEX A/S']

Genmab_list = ['Genmab A/S', 'Genmab', 'Genmab AS', 'Genmab A/S,']

Gabriel_Holding_list = ['GABRIEL A/S','Gabriel A/S','Gabriel'] 

G4S_plc_list = ['G4S Security Services A/S',
 'G4S',
 'G4S SECURITY SERVICES A/S',
 'G4S Security Services',
 'G4S SECURITY SERVICES A/S - Århus',
 'G4S SECURITY SERVICES A/S Billetbetaling',
 'G4S Vagt A/S',
 'G4S Vagt',
 'G4S vagt A/S',
 'G4S Security Service A/S',
 'G4S VAGT A/S']

Fynske_Bank_list = ['Fynske Bank A/S',
 'Fynske Bank Nyborg Afd.',
 'Fynske Bank Assens Afd.',
 'Fynske Bank Fredericia Afd.',
 'Fynske Bank',
 'Fynske Bank Assens',
 'Fynske Bank Børkop',
 'Fynske Bank Svendborg',
 'Leasing Fyn Bank A/S',
 'Leasing Fyn',
 'LEASING FYN BANK A/S',
 'Leasing Fyn & Factoring Bankaktieselskab',
 'LEASING FYN & FACTORING BANKAKTIESELSKAB',
 'LEASING FYN & FACTORING BANKAKTIESELSKAB.']

Flügger_group_list = ['Flügger A/S','FLÜGGER FARVER',
 'Flügger','Flügger Farver',
 'Flügger farver Årslev',
 'Flügger A/s',
 'Flügger A/S, Salgsafdeling A086 Ringkøbing',
 'FLÜGGER A/S, SALGSAFDELING',
 'FLüGGER A/S, SALGSAFDELING',
 'FLÜGGER A/S',
 'Flügger farver',
 'Flügger i Ørbæk v/Mette Marie Stokholm',
 'FLÜGGER FARVER V/MALERMESTER PAUL ISVIK',
 'Flügger HR',
 'Flügger farver, Frank Kristensen, Ringparkens Butikstorv',
 'Flügger farver A/S',
 'FLÜGGER A/S SALGSAFDELING',
 'Flügger Group',
 'FLÜGGER A/S Salgsafdeling',
 'Flügger farvehandel',
 'FLÜGGER Farver']

FLSmidth_list = ['FLSmidth',
 'FLSmidth, Inc.',
 'FLSmidth Private Limited',
 'FLSMIDTH & CO. A/S',
 'FLSmidth & Co. A/S',
 'flsmidth-inc',
 'Flsmidth A/s',
 'Flsmidth & Co. A/s',
 'FLSmidth HAMBURG GmbH',
 'FLSmidth A/S',
 'FLSmidth Group Communications',
 'Flsmidth A/S',
 'FLSMIDTH A/S',
 'FLSmidth Airtech',
 'FLSmidth Airloq',
 'FLSmidth Automation',
 'FLSmidth Airloq A/S',
 'FLSmidth Minerals A/S']

FirstFarms_list = ['FirstFarms A/S']

Fast_Ejendom_Danmark_list = ['Fast Ejendom Danmark A/S']

DSV_Panalpina_list = ['DSV',
 'DSV Transport A/S',
 'DSV Road A/S',
 'DSV A/S',
 'DSV Brøndby',
 'DSV Esbjerg',
 'DSV Miljø A/S',
 'DSV Solutions',
 'DSV Miljø',
 'DSV Road',
 'DSV Group IT',
 'DSV Air & Sea A/S',
 'DSV ROAD A/S',
 'DSV Road Holding A/S',
 'Støvring Biltransport DSV Road A/S',
 'DSV Air & Sea',
 'DSV Road A/S, Carlsberg',
 'DSV TRANSPORT A/S',
 'DSV Solutions A/S']


Drilling_Company_of_1972_list = ['Maersk Drilling A/S',
 'Maersk Drilling',
 'A.P. Møller - Mærsk A/S (Maersk Drilling)',
 'Maersk Drilling Services A/S',
 'Mærsk Drilling',
 'A. P. Møller Mærsk Drilling',
 'A.P. Møller - Mærsk - Drilling SAP',
 'A.P. Møller - Mærsk - Drilling',
 'Maersk Drilling Norge A/S',
 'MÆRSK Drilling']

Djurslands_Bank = ['Djurslands Bank',
 'Djurslands Bank A/S',
 'DJURSLANDS BANK',
 'DJURSLANDS BANK A/S ALLINGAABRO AFD.',
 'DJURSLANDS BANK A/S GRENAA AFDELING',
 'DJURSLANDS BANK A/S KOLIND AFDELING',
 'DJURSLANDS BANK A/S']


DFDS_list = ['DFDS',
 'DFDS A/S',
 'DFDS AS',
 'Dfdsseaways',
 'DFDS Group',
 'DFDS a/s',
 'DFDS House',
 'DFDS Canal Tours',
 'DFDS Canal Tours A/S',
 'DFDS Seaways A/S',
 'DFDS Seaways',
 'DFDS Transport A/S, dsv',
 'DFDS TRANSPORT A/S',
 'DFDS Transport A/S',
 'DFDS CANAL TOURS A/S',
 'DFDS Tor Line']

Demant_list = ['Demant',
 'William Demant Group',
 'William Demant',
 'William Demant Holding A/S',
 'WILLIAM DEMANT HOLDING A/S',
 'William Demant Holding',
        'Interacoustics A/S', 'INTERACOUSTICS A/S',
          'Oticon',
 'Oticon Medical',
 'Oticon A/S',
 'OTICON A/S',
          'Audika',
          'Bernafon', 'Bernafon - Phonic Ear A/S'
]

Dantax_list = [
 'Dantax Radio A/S',
 'Dantax',
 'Dantax Radio',
 'Dantax Steuerberatungsgesellschaft mbH',
 'Dantax Steuerberatungsgesellschaft mbh',
 'DANTAX Steuerberatungsgesellschaft mbH']

Danske_Bank_list = ['Danske Bank',
 'Danske Bank A/S',
 'Danske Bank A/s',
 'Danske Bank-koncernen',
 'Danske Bank Group',
 'DANSKE BANK A/S',
 'Danske Bank IT',
 'Danske Bank (.com)',
 'Den Danske Bank A/S',
 'Den Danske Bank','Danske Invest Administration A/S', 'Danske Invest',
              'Realkredit Danmark','Realkredit Danmark A/S','home a/s','home a/s',
 'Home A/S',
 'Home A/s','Danica Pension','MobilePay', 'Mobilepay']


Danske_Andelskassers_Bank_list = ['Danske Andelskassers Bank A/S',
 'Danske Andelskassers Bank AS',
 'Danske Andelskassers Bank A/S - Augustenborg',
 'Danske Andelskassers Bank',
 'DANSKE ANDELSKASSERS BANK A/S']


D_S_Norden_list = ['Dampskibsselskabet NORDEN A/S',
 'Dampskibsselskabet NORDEN',
 'Dampskibsselskabet Norden A/S',
 'DAMPSKIBSSELSKABET NORDEN A/S',
                  
                  'Norient Product Pool ApS']


Copenhagen_Capital_list = ['Copenhagen Capital A/S']


Columbus_list = ['Columbus A/S',
 'Columbus Danmark A/S',
 'Columbus',
 'COLUMBUS A/S',
 'Columbus A/s']

Coloplast_B_list = ['Coloplast A/S',
 'Coloplast Danmark A/S',
 'Coloplast',
 'Coloplast Ltd',
 'COLOPLAST A/S',
 'Coloplast Denmark and Emerging Markets']


Chr_Hansen_Holding_list = ['Chr. Hansen Holding A/S']

ChemoMetec_list = ['ChemoMetec A/S', 'CHEMOMETEC A/S', 'Chemometech', 'Chemometec A/S']

cBrain_list = ['cBrain A/S',
 'cBrain',
 'CBrain A/S',
 'CBRAIN',
 'cBrain Technology',
 'cBrain Technology Copenhagen']

Carlsberg_list = ['Carlsberg Danmark',
 'Carlsberg Breweries A/S',
 'Carlsberg Group',
 'Carlsberg Group A/S',
 'Carlsberg Danmark A/S',
 'Visit Carlsberg',
 'Carlsberg',
 'Carlsberg Group Procurement AG',
 'Carlsberg A/S Ejendomme',
 'CARLSBERG EJENDOMME HOLDING A/S',
 'Carlsberg Danmark AS',
 'CARLSBERG BREWERIES A/S',
 'Carlsbergfondet',
 'Carlsberg IT A/S',
 'CARLSBERG DANMARK A/S C/O PERSONALE',
 'Carlsberg A/S',
 'Carlsberg Group HR',
 'Carlsberg Danmark A/S Tuborg FB/Terminal Vest/Fredericia',
 'Carlsberg Danmarks',
 'CARLSBERG Danmark A/S Depot Holstebro',
 'CARLSBERG DANMARK A/S Depot Århus',
 'CARLSBERG Danmark A/S Depot Næstved']

Brøndby_IF = ['Brøndby IF Arrangementer','Brøndbyernes IF'] # dobbeltjek

brdr_hartmann_list = ['Brødrene Hartmann A/S', 'Brødrene Hartmann', 'BRØDRENE HARTMANN A/S']

Brødrene_A_O_Johansen_list = ['Brødrene A & O Johansen A/S',
 'Brødrene A&O Johansen A/S',
 'Brødrene A. & O. Johansen A/S',
 'Brødrene A&O Johansen',
 'BRØDRENE A. & O. JOHANSEN A/S',
 'Brødrene A & O Johansen A/S, Hovedstaden',
 'Brødrene A. & O. Johansen A/s',
 'AO - Brødrene A & O Johansen A/S',
 'Brødrene A & O Johansen',
 'Brødrene A & O Johansen A/S Nykøbing F',
 'BRØDRENE A & O JOHANSEN A/S',
 'Brødrene Ao Johansen',
]



Boozt_list = ['Boozt Fashion',
 'Boozt',
 'Boozt Fashion AB',
 'Boozt.com']

Boliga_Gruppen_list = ['Boliga Gruppen A/S','Boliga','Boliga ApS','Boliga Aps', 'Boliga.dk']

blue_vision_list = ['Contra A/S','CONTRA A/S','Contra - Hillerød',
 'Contra - Helsingør','ConTra International','Contra - Viborg','ConTra Denmark','Contra','ConTra']

BioPorto_list = ['BioPorto A/S',
 'BioPorto Diagnostics A/S',
 'BioPorto Diagnostics',
 'Bioporto Diagnostics A/S',
 'BIOPORTO A/S',
 'Bioporto A/S']

Bavarian_Nordic_list = ['Bavarian Nordic A/S', 'Bavarian Nordic', 'BAVARIAN NORDIC A/S']

BankNordik_list = ['BankNordik',
 'P/F Banknordik',
 'P/F Banknordik Lyngby',
 'Banknordik',
 'BANKNORDIK']

Bang_Olufsen_list = ['Bang & Olufsen A/S',
 'Bang & Olufsen America Inc.',
 'Bang & Olufsen a/s',
 'Bang & Olufsen Operations',
 'Bang & Olufsen Operations a/s','B&O PLAY']

Atlantic_Petroleum = ['Atlantic Petroleum P/F']

Ambu_list = ['Ambu A/S','Ambu',
 'Ambu Denmark','Ambu A/s','AMBU A/S']


Alm_Brand_list = ['Alm. Brand',
 'Alm. Brand A/S',
 'Alm. Brand Forsikring A/S',
 'Alm. Brand Forsikring A/s',
 'Alm. Brand A/s',
 'Alm. Brand Forsikring',
 'Alm. Brand Bank A/S',
 'Alm. Brand Leasing A/S',
 'Alm. Brand Bank',
 'Alm. Brand Markets',
 'Alm. Brand Asset Management',
 'Alm. Brand BANK',
 'ALM. BRAND A/S',
 'Alm. Brand Region Syd',
 'ALM. BRAND FINANS A/S',
 'ALM. BRAND BANK',
 'ALM. Brand',
 'Alm. Brand Finans',
 'ALM. BRAND',
 'Alm. Brand Henton',
 'ALM. BRAND REGION SYD']


ALK_Abello_list =['ALK-Abelló A/S',
 'Alk-Abelló A/S',
 'ALK-Abelló',
 'ALK-Abelló Arzneimittel GmbH',
 'ALK-abelló','ALK Abello'
 'ALK-Abello',
 'Alk-Abello A/S',
 'ALK-Abello A/S']

AGF_list = ['AGF A/S', 'AGF','AGF Fodbold','AGF Håndbold','AGF Svømning','AGF Tennis']
AaB_list = ['AaB esport', 'AaB A/S','Aalborg Boldspilklub A/S', 'AALBORG BOLDSPILKLUB AF 1885']

In [28]:
list_of_names = ["Københavns Lufthavne",'Kreditbanken','Jutlander Bank','Jeudan','ISS',"InterMail",'Hvidbjerg Bank','HusCompagniet','Harboes Bryggeri B',
                'H+H International','Gyldendal B','Grønlandsbanken','GreenMobility','GN Store Nord','Glunz & Jensen Holding','Genmab','Gabriel Holding'
                ,'G4S plc','Fynske Bank','Flügger group B','FLSmidth & Co.','FirstFarms','Fast Ejendom Danmark','DSV Panalpina','Drilling Company of 1972',
                'Djurslands Bank','DFDS','Demant','Dantax','Danske Bank','Danske Andelskassers Bank','D/S Norden','Copenhagen Capital','Columbus','Coloplast B',
                'Chr. Hansen Holding', 'ChemoMetec','CBRAIN','Carlsberg','Brøndby IF','Brdr.Hartmann','Brdr. A & O Johansen præf.','Boozt','Boliga Gruppen',
                'Blue Vision A','BioPorto','Bavarian Nordic','BankNordik','Bang & Olufsen','Atlantic Petroleum','Ambu','Alm. Brand','ALK-Abelló B','AGF B',
                'AaB']
list_of_lists = [Københavns_Lufthavne_list,Kreditbanken_list,Jyske_Bank_list,Jutlander_Bank_list,Jeudan_list,ISS_list,InterMail_list,Hvidbjerg_Bank_list,HusCompagniet_list,
                Harboes_Bryggeri_list,H_H_International_list,Gyldendal_list,Grønlandsbanken_list, GreenMobility_list,GN_Store_Nord_list,Glunz_and_Jensen_Holding_list,
                Genmab_list,Gabriel_Holding_list,G4S_plc_list,Fynske_Bank_list,Flügger_group_list,FLSmidth_list,FirstFarms_list,Fast_Ejendom_Danmark_list,DSV_Panalpina_list,
                Drilling_Company_of_1972_list,Djurslands_Bank,DFDS_list,Demant_list,Dantax_list,Danske_Bank_list,Danske_Andelskassers_Bank_list,D_S_Norden_list,Copenhagen_Capital_list,
                Columbus_list,Coloplast_B_list,Chr_Hansen_Holding_list,ChemoMetec_list,cBrain_list ,Carlsberg_list,brdr_hartmann_list,Brødrene_A_O_Johansen_list,Boozt_list,
                Boliga_Gruppen_list,blue_vision_list,BioPorto_list,Bavarian_Nordic_list,BankNordik_list,Bang_Olufsen_list,Atlantic_Petroleum,Ambu_list,Alm_Brand_list,
                ALK_Abello_list,AGF_list,AaB_list]

In [29]:
for firm in list(zip(list_of_names,list_of_lists)):
    print(firm)

('Københavns Lufthavne', ['Københavns Lufthavne A/S', 'Københavns Lufthavn', 'Københavns Lufthavne (CPH)', 'Københavns lufthavne', 'Copenhagen Airport Hotels A/S', 'COPENHAGEN AIRPORT HOTEL ApS', 'COPENHAGEN AIRPORTS INTERNATIONAL A/S', 'Københavns Lufthavne A/S, Hrm'])
('Kreditbanken', ['Kreditbanken A/S', 'Kreditbanken A/S Sønderborg', 'Kreditbanken A/S Tønder', 'Kreditbanken', 'Kreditbanken i Sønderborg', 'KREDITBANKEN A/S'])
('Jutlander Bank', ['Jyske Bank', 'Jyske Finans', 'JYSKE FINANS A/S', 'Jyske Invest'])
('Jeudan', ['Jutlander Bank A/S', 'Jutlander Bank', 'Jutlander Bank AS', 'JUTLANDER BANK A/S', ' Jutlander Bank A/S'])
('ISS', ['Jeudan A/S', 'Jeudan', 'JEUDAN A/S', 'Jeudan Servicepartner A/S', 'JEUDAN SERVICEPARTNER A/S', 'Jeudan Servicepartner'])
('InterMail', ['ISS A/S', 'ISS World Services A/S', 'ISS World', 'ISS Facility Services A/S', 'ISS Facility Services AS', 'Personalehuset - Part of ISS Facility Services A/S', 'HR afd. i ISS Facility Services A/S', 'Medarbejderfon